In [1]:
import os
# from itertools import cycle
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow import keras
# from tensorflow.keras import backend
# from tensorflow.keras import models
from tensorflow.keras.metrics import Recall, Precision, BinaryAccuracy, FalsePositives, \
                                    TruePositives, TrueNegatives, FalseNegatives, AUC, Metric
import sys
sys.path.insert(1, "../bin/")
from metrics_helper import MatthewsCorrelationCoefficient
from model_definition import load_compile_model
from test_zoonosis_model import load_test_data
# from tensorflow.python.keras.utils import metrics_utils
# from tensorflow.python.keras.utils.generic_utils import to_list
from tensorflow.keras.utils import plot_model
from sklearn.metrics import plot_roc_curve, precision_recall_curve, f1_score, roc_auc_score, \
                            roc_curve, matthews_corrcoef, confusion_matrix, auc
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
# import hiplot as hip

# import plotly.graph_objects as go
# from mlxtend.plotting import plot_confusion_matrix
%matplotlib inline

# from tf_keras_vis.saliency import Saliency
# from tf_keras_vis.scorecam import Scorecam
# from tf_keras_vis.gradcam_plus_plus import GradcamPlusPlus

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, enable=True) 

In [ ]:
# main_dir = "/scratch2/users/rserage/"
main_dir = "/home/rserage/Documents/Masters Project/Docs"
test_dirs = os.listdir(main_dir)

test_dirs = [os.path.join(main_dir, fol) for fol in test_dirs]

In [ ]:
test_dirs = list(filter(lambda x: x.__contains__("Zoo"), test_dirs)) # "Data" for dataset

test_dirs

In [ ]:
# test_dirs = ["/scratch3/users/rserage/ZoonosisFull/"]
# model_dirs = ["/users/rserage/Zoon0PredV/notebooks/"]

In [ ]:
models = [os.path.join(path, 'model') for path in test_dirs]
test_data = [os.path.join(path, 'test') for path in test_dirs]

models

In [ ]:
main = pd.DataFrame()
PRS = dict()
ROCS = dict()

for model_, test_data in zip(models, test_data):
    model = load_compile_model(model_)
    metrics, PR, ROC = test_model(model, test_data)
    main = pd.concat([main, metrics], sort=False)
    PRS.update(PR)
    ROCS.update(ROC)

In [ ]:
ROCS

In [ ]:
main

In [ ]:
PRS

In [ ]:
ROC_curve(main, ROCS)

In [ ]:
PR_curve(main, PRS)

In [ ]:
model = load_compile_model("/users/rserage/Zoon0PredV/notebooks/model/")

In [ ]:
dot_img_file = "model_1.png"
plot_model(model, to_file=dot_img_file, show_layer_names=False, show_shapes=False)

In [ ]:
# data = [{
#     'epoch': idx,
#     'loss': history.history['loss'][idx],
#     'val_loss': history.history['val_loss'][idx],
#     'accuracy': history.history['accuracy'][idx],
#     'val_accuracy': history.history['val_accuracy'][idx],
#     'precision': history.history['precision'][idx],
#     'val_precision': history.history['val_precision'][idx],
#     'recall': history.history['recall'][idx],
#     'val_recall': history.history['val_recall'][idx]
# } 
#     for idx in range(0, 10)] 
df = pd.read_csv('logs.csv')
data = df.to_dict('records')

hip.Experiment.from_iterable(data).display()

In [ ]:
# test_data_iterator.reset()
for _ in range(5):
    img, label = test_data_iterator.next()
    print(img.shape)   #  (1,256,256,3)
    plt.figure(figsize=(12,6))
    plt.imshow(img[0], cmap=plt.cm.binary)
    # plt.show()
    # print(img[0])
    break

In [ ]:
# model = tf.keras.models.load_model('model', custom_objects={"MCC": MatthewsCorrelationCoefficient})
# model.summary()

In [ ]:
img_path = '/scratch3/users/rserage/ZoonosisData/test/human_false_A0A023NBJ1_Autoprotease.png'

# img_path = 'testing.png'

img = load_img(img_path, target_size=(128, 128), color_mode='grayscale')
img_tensor = img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.

In [ ]:
img_tensor, _ = test_data_iterator.next()

In [ ]:
img_tensor = img_tensor[0]

In [ ]:
img_tensor = img_tensor.reshape((1,128,128,1))

In [ ]:
img_tensor.shape

In [ ]:
plt.figure(figsize=(12,6))

plt.imshow(img_tensor[0], cmap=plt.cm.binary)
plt.axis("off")
# plt.show()

In [ ]:
model.input

In [ ]:
# Extracts the outputs of the top 8 layers:
layer_outputs = [layer.output for layer in model.layers[:]]
# Creates a model that will return these outputs, given the model input:
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)

In [ ]:
activations = activation_model.predict(img_tensor)

In [ ]:
first_layer_activation = activations[0]

first_layer_activation.shape

In [ ]:
plt.figure(figsize=(12,6))

plt.matshow(first_layer_activation[0, :, :, 3], cmap='jet')
plt.axis("off")
# plt.show()

In [ ]:
activations

In [ ]:
# Names of the layers, so can have them as part of our plot
layer_names = [ layer.name for layer in model.layers[:] ]
    
images_per_row = 16

# Display feature maps
for layer_name, layer_activation in zip(layer_names[1:-2], activations[1:-2]):
    # Number of features in the feature map
    n_features = layer_activation.shape[-1]

    # The feature map has shape (1, size, size, n_features)
    size = layer_activation.shape[1]

    # Tile the activation channels matrix
    n_cols = n_features // images_per_row
    display_grid = np.zeros((size * n_cols, images_per_row * size))

    # Tile each filter into horizontal grid
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0, :, :, col * images_per_row + row]
            # Post-process the feature to make it visually palatable
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size, row * size : (row + 1) * size] = channel_image

    # Display the grid
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    
# plt.show()

In [ ]:
import innvestigate
import innvestigate.utils

In [ ]:
from attention_mapping import load_samples

In [ ]:
base_path = "../bin/"
list_path_to_images = [
    os.path.join(base_path, img_path) 
    for img_path in 
    list(
        filter(lambda File: File.endswith(".png"), os.listdir(base_path))
    )
]
list_path_to_images

In [ ]:
images, X = load_samples(list_path_to_images)

In [ ]:
model = load_compile_model('../model')

In [ ]:
X.shape

In [ ]:
model.predict(X).tolist()

In [ ]:
df = pd.read_csv("/home/rserage/Documents/Masters Project/Docs/Drafts/Thesis/assets/figures/adoring_knuth/proof_of_concept_data.csv")

In [ ]:
df['Entry'].tolist()

In [ ]:
df['Virus hosts'].str.split(";").apply(len)

In [ ]:
dot_img_file = "Zoon0Pred_model_architecture.png"
plot_model(model, to_file=dot_img_file, show_layer_names=False, show_shapes=False)

In [ ]:
# Strip softmax layer
# model = innvestigate.utils.model_wo #.model_wo_softmax(model)

# Create analyzer
analyzer = innvestigate.create_analyzer("deep_taylor", model)

# Add batch axis and preprocess
# x = preprocess(image[None])
# Apply analyzer w.r.t. maximum activated output-neuron
a = analyzer.analyze(X)

# Aggregate along color channels and normalize to [-1, 1]
a = a.sum(axis=np.argmax(np.asarray(a.shape) == 3))
a /= np.max(np.abs(a))
# Plot
plt.imshow(a[0], cmap="seismic", clim=(-1, 1))

In [ ]:
df = pd.read_csv("~/Documents/Masters Project/Docs/Drafts/Thesis/assets/figures/adoring_knuth/Zoon0PredV/Zoon0PredV_trainingLogs.csv")

In [ ]:
df.columns

In [ ]:
df.head(12)

In [ ]:
df.sort_values("MCC", ascending=False)

In [ ]:
df.set_index('epoch', inplace=True)

In [ ]:
df['binary_accuracy'] = df['binary_accuracy'] * 100
df['val_binary_accuracy'] = df['val_binary_accuracy'] * 100

In [ ]:
plt.style.use('fivethirtyeight')

In [ ]:
sns.set_context('paper')

In [ ]:
import matplotlib

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(8,5), dpi=300, sharex=True)

df[['binary_accuracy', 'val_binary_accuracy']].plot(kind='line', ax=ax1)
ax1.get_legend().remove()
# ax1.legend(None)#['Training accuracy', 'Validation accuracy'], loc='lower right', fontsize=8)
# ax1.get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
# ax1.set_yticklabels(ax1.get_yticklabels(), fontsize=5)
ax1.set_ylim(50, 100)

ax1.set_ylabel('Accuracy')#, fontsize=10, fontweight='bold', rotation=0)
# ax1.yaxis.set_label_coords(-0.073, 0.9)
ax1.set_title('A', fontdict={'fontsize':10, 'fontweight':'bold'}, loc='left')

df[['loss', 'val_loss']].plot(kind='line', ax=ax2)
ax2.legend(['Training', 'Validation'], fontsize=8, bbox_to_anchor=(0.98, 1.25))
ax2.set_ylim(0.0, 2.0)
# ax2.set_yticklabels(ax2.get_yticklabels(), fontsize=5)
ax2.set_title('B', fontdict={'fontsize':10, 'fontweight':'bold'}, loc='left')

ax2.set_ylabel('Loss') #, fontsize=10, fontweight='bold', rotation=0)
# ax2.yaxis.set_label_coords(-0.075, 0.9)

# plt.subplots_adjust(wspace=0.05)

plt.savefig("/home/rserage/Documents/Masters Project/Docs/Drafts/Thesis/assets/figures/accuracy_loss.png")

In [ ]:
vals = {'': 7299,
 'KW-1160': 7299,
 'NCBI virus': 6271,
 'Virus-Host DB': 3172,
 'EID2': 3166}

In [ ]:
list(vals.values())

In [ ]:
# Bring some raw data.
frequencies = list(vals.values()) #[6, 16, 75, 160, 244, 260, 145, 73, 16, 4, 1]
# In my original code I create a series and run on that,
# so for consistency I create a series from the list.
freq_series = pd.Series(frequencies)

x_labels = list(vals.keys())

# Plot the figure.
plt.figure(figsize=(5, 3.5), dpi=300)
ax = freq_series.plot(kind="bar")
ax.get_children()[0].set_color('g')
# ax.set_title("Amount Frequency")
# ax.set_xlabel("Dataset")
ax.set_ylabel("No. of records", fontsize=7)
ax.set_xticklabels(x_labels, rotation=0, fontsize=5)
ax.set_yticklabels('', rotation=0, fontsize=7)
# ax.set_ylim(0, 8000)
ax.xaxis.grid(False)
rects = ax.patches

# Make some labels.
labels = frequencies #[f"label{i}" for i in range(len(rects))]

for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(
        rect.get_x() + rect.get_width() / 2, height + 5, label, ha="center", va="bottom", fontsize=7
    )

plt.savefig("/home/rserage/Documents/Masters Project/Docs/Drafts/Thesis/assets/figures/tracking.png")

In [4]:
# import numpy as np
# import pandas as pd
# from tensorflow import keras
# import sys
# sys.path.insert(1, "../bin/")
# from model_definition import load_compile_model
from test_zoonosis_model import load_test_data

In [3]:
physical_devices

[]

In [4]:
tf.__version__

'2.6.0'

In [5]:
model = load_compile_model('../model')

In [6]:
directory = "../Zoon0PredV/test/" # os.path.join(folder, 'test'))

data_generator = ImageDataGenerator(rescale=1./255)
test_data = data_generator.flow_from_directory(
    directory,
    target_size=(224, 224),
    color_mode='grayscale',
    classes=['human-false', 'human-true'],
    class_mode='binary',
    batch_size=64,
    shuffle=False,
    seed=19980603
)

steps = int(round(test_data.n / 64))

Found 19326 images belonging to 2 classes.


In [7]:
predictions = (model.predict(test_data, verbose=1)).astype(np.float32).reshape((-1,)).tolist()

302/302 [==============================] - 631s 2s/step


In [12]:
test_data.reset()
evals = model.evaluate(test_data, verbose=1, return_dict=True, steps=steps)

302/302 [==============================] - 22s 69ms/step - loss: 0.0979 - binary_accuracy: 0.9683 - FP: 259.0000 - TP: 11245.0000 - TN: 7469.0000 - FN: 353.0000 - MCC: 0.9342 - AUC: 0.9931 - Recall: 0.9696 - Precision: 0.9775s - loss: 0.0947 - binary_accuracy: 0.9694 - FP: 259.0000 - TP: 9345.0000 - TN: 7469.0000 - FN: 271.0000 - MCC: 0.9382 - AUC: 


In [8]:
test_data.reset()
true_values = test_data.labels.astype(np.float32).reshape((-1,)).tolist()

In [9]:
len(test_data.filenames)

19326

In [10]:
test_data.reset()
labels = test_data.filenames #.reshape((-1,)).tolist()

In [11]:
# Creating dataframe automatically sorts out issue of dimentions of numpy arrays
results = pd.DataFrame.from_dict({
    'Filenames':labels,
    'True values':true_values,
    'Predictions':predictions}
)

In [12]:
results['Entry'] = results["Filenames"].str.split("_").apply(lambda val: val[2])

In [13]:
# zoonopred_data = "/home/rserage/Documents/Masters Project/Docs/Drafts/Thesis/assets/figures/adoring_knuth/Zoon0PredV/Zoon0PredVData.csv.gz"
zoonopred_data = "../Zoon0PredV/Zoon0PredVData.csv.gz"

In [14]:
df = pd.read_csv(zoonopred_data)

## False positives

In [15]:
results.rename({'Predictions': "Probas"}, axis=1, inplace=True)
results['Predictions'] = results['Probas'].apply(lambda val: 1.0 if (val >= 0.5) else 0.0)

In [16]:
results

,Filenames,True values,Probas,Entry,Predictions
0,human-false/human_false_A0A023HYP5_Capsid.png,0.0,0.034407,A0A023HYP5,0.0
1,human-false/human_false_A0A023HYQ5_Capsid.png,0.0,0.033788,A0A023HYQ5,0.0
2,human-false/human_false_A0A023HZM1_Capsid.png,0.0,0.016890,A0A023HZM1,0.0
3,human-false/human_false_A0A023IRJ9_Genome.png,0.0,0.007470,A0A023IRJ9,0.0
4,human-false/human_false_A0A023J5A4_Pre_glycopr...,0.0,0.372747,A0A023J5A4,0.0
...,...,...,...,...,...
19321,human-true/human_true_X5F5S8_Major.png,1.0,0.582696,X5F5S8,1.0
19322,human-true/human_true_X5FNQ6_Major.png,1.0,0.803354,X5FNQ6,1.0
19323,human-true/human_true_X5FPI3_Major.png,1.0,0.852707,X5FPI3,1.0
19324,human-true/human_true_X5FQL4_Major.png,1.0,0.814458,X5FQL4,1.0


In [17]:
false_positives = results[(results['True values'] != results['Predictions']) & (results['Predictions'] == 1.0)].copy()
print(false_positives.shape[0])
false_positives.head()

259


,Filenames,True values,Probas,Entry,Predictions
7,human-false/human_false_A0A023NBJ1_Autoproteas...,0.0,0.611747,A0A023NBJ1,1.0
69,human-false/human_false_A0A060BCT2_Capsid.png,0.0,0.750553,A0A060BCT2,1.0
70,human-false/human_false_A0A060BLI4_Capsid.png,0.0,0.660651,A0A060BLI4,1.0
71,human-false/human_false_A0A060BM97_Capsid.png,0.0,0.687383,A0A060BM97,1.0
82,human-false/human_false_A0A067YSB8_Envelope.png,0.0,0.803354,A0A067YSB8,1.0


In [32]:
entries_false_positives = df[df['Entry'].isin(false_positives['Entry'].tolist())].copy()

In [33]:
entries_false_positives['Protein'].nunique()

78

In [39]:
entries_false_positives['Virus hosts'].nunique()

136

In [74]:
df[(df['Protein'].str.contains("[Pp]ortal")) & (df['Infects human'].str.contains("false"))]

,Entry,Protein,Length,Species name,Species taxonomic ID,Species family,Virus hosts,Virus hosts ID,Host kingdom,Host superkingdom,Molecule type,Infects human
30,A0A023MH03,Portal protein,405.0,Escherichia virus FFH1,2049893,Demerecviridae,Escherichia coli [TaxID: 562],562,Escherichia coli,Bacteria,unknown,human-false
162,A0A023W6G5,Portal protein,517.0,Serratia virus PS2,2846184,Myoviridae,Serratia marcescens [TaxID: 615],615,Serratia marcescens,Bacteria,unknown,human-false
171,A0A023ZV09,Portal protein,524.0,Escherichia virus E112,1914201,Myoviridae,Escherichia coli [TaxID: 562],562,Escherichia coli,Bacteria,unknown,human-false
174,A0A023ZX00,Portal protein,524.0,Escherichia virus T4,10665,Myoviridae,Escherichia coli [TaxID: 562],562,Escherichia coli,Bacteria,unknown,human-false
178,A0A024AZY6,Portal protein,557.0,Bacillus virus CAM003,1918012,Herelleviridae,Bacillus thuringiensis [TaxID: 1428],1428,Bacillus thuringiensis,Bacteria,unknown,human-false
...,...,...,...,...,...,...,...,...,...,...,...,...
38719,X2KR22,"Phage portal protein, HK97 family",390.0,Campylobacter phage CJIE4-1,1470458,Siphoviridae,Campylobacter jejuni [TaxID: 197],197,Campylobacter jejuni,Bacteria,unknown,human-false
38720,X2KR48,"Phage portal protein, HK97 family",390.0,Campylobacter phage CJIE4-3,1470460,Siphoviridae,Campylobacter jejuni [TaxID: 197],197,Campylobacter jejuni,Bacteria,unknown,human-false
38721,X2KS29,HK97 family portal protein,383.0,Enterococcus virus AUEF3,2560417,Siphoviridae,Enterococcus faecalis [TaxID: 1351],1351,Enterococcus faecalis,Bacteria,unknown,human-false
38724,X2KXI2,"Phage portal protein, HK97 family",390.0,Campylobacter phage CJIE4-4,1470461,Siphoviridae,Campylobacter jejuni [TaxID: 197],197,Campylobacter jejuni,Bacteria,unknown,human-false


Helicobacter pylori - interesting
Yersinia enterocolitica - interesting

5 viruses were falsely classified as positive and al for the portal protein and 4 belonged had bacterial hosts and only one had a plant host. It is unclear why this is but it may warrant investigation of the portal protein. Portal proteins have low sequence similarity, but conserved function, however, they are said to be plastic and researched as a potential antiviral drug target in HSV-1 and HSV-2.

In [58]:
results[(results['Filenames'].str.contains("Portal")) & (results['Filenames'].str.contains("true")) ]

,Filenames,True values,Probas,Entry,Predictions


In [56]:
entries_false_positives[entries_false_positives['Protein'].str.contains("Portal protein")]

,Entry,Protein,Length,Species name,Species taxonomic ID,Species family,Virus hosts,Virus hosts ID,Host kingdom,Host superkingdom,Molecule type,Infects human
5038,A0A0K2FHA1,Portal protein,536.0,Achromobacter phage phiAxp-2,1664246,Siphoviridae,Achromobacter xylosoxidans [TaxID: 85698],85698,Achromobacter xylosoxidans,Bacteria,unknown,human-false
22756,A7TWJ1,Portal protein,278.0,Staphylococcus virus tp310-2,2846382,Siphoviridae,Staphylococcus aureus [TaxID: 1280],1280,Staphylococcus aureus,Bacteria,unknown,human-false
29330,I7HHN4,Portal protein,602.0,Helicobacter virus KHP30,1987149,Podoviridae,Helicobacter pylori [TaxID: 210],210,Helicobacter pylori,Bacteria,unknown,human-false
29343,I7KR94,Portal protein,525.0,Yersinia virus R1RT,1921705,Myoviridae,Yersinia enterocolitica [TaxID: 630],630,Yersinia enterocolitica,Bacteria,unknown,human-false
31851,M4QNQ7,Portal protein,432.0,Tetraselmis viridis virus S20,754070,Siphoviridae,Tetraselmis viridis [TaxID: 748148],748148,Viridiplantae,Eukaryota,unknown,human-false


- A total of 36 entries with 19 diff protein names which were falsely classified from bacterial hosts. Some had uncharacterised proteins which shared no structural identity with any protein on uniprot.

- The rest may be due to similarity of proteins and shared by multiple hosts. such viruses may have human interacting partners but may lack machinery for replication which may be identified by other PPI studies.

In [65]:
entries_false_positives[entries_false_positives['Host superkingdom'].str.contains("[Bb]acteri")]['Protein'].nunique()

19

In [70]:
entries_false_positives.loc[31986, :]

Entry                                                              M9ZRW9
Protein                                                Genome polyprotein
Length                                                               3010
Species name                                                  Pegivirus B
Species taxonomic ID                                              1307801
Species family                                               Flaviviridae
Virus hosts             Chaerephon [TaxID: 242383]; Epomophorus labiat...
Virus hosts ID          584996; 258867; 907075; 110943; 242383; 903567...
Host kingdom                                                      Metazoa
Host superkingdom                                               Eukaryota
Molecule type                                                    ssRNA(+)
Infects human                                                 human-false
Name: 31986, dtype: object

In [68]:
entries_false_positives['Virus hosts'].apply(lambda items: len(items.split("; "))).sort_values()

31        1
23079     1
23294     1
23969     1
23971     1
         ..
35184    14
14394    16
31990    19
31986    19
3536     66
Name: Virus hosts, Length: 259, dtype: int64

In [40]:
entries_false_positives['Species name'].nunique()

167

In [43]:
entries_false_positives['Species name'].unique()

array(['Pestivirus Giraffe-1', 'Bovine astrovirus', 'Pegivirus A',
       'Lizard atadenovirus A', 'Citrobacter virus Miller',
       'Pipapillomavirus 1', 'Tursiops aduncus papillomavirus',
       'Bovine polyomavirus 2', 'Cotia virus', 'Equid alphaherpesvirus 1',
       'Israel turkey meningoencephalomyelitis virus',
       'Papio cynocephalus polyomavirus 2',
       'Avon-Heathcote Estuary associated circular virus 15',
       'Bat paramyxovirus', 'Simian immunodeficiency virus',
       'Murine respirovirus', 'Bovine respirovirus 3',
       'Brucella virus Tb', 'Necocli orthohantavirus',
       'Dromedary astrovirus', 'Achromobacter phage phiAxp-2',
       'Simian adenovirus 19', 'Simian mastadenovirus E',
       'Simian adenovirus 8', 'Salmonella virus P22', 'Entebbe bat virus',
       'Hepatovirus G', 'Hepatovirus F', 'Deltapapillomavirus 5',
       'Porcine astrovirus 3', 'Pseudomonas phage O4',
       'Gordonia phage McGonagall', 'Bat mastadenovirus C',
       'Teschovirus A', '

## False negatives

In [20]:
results[(results['True values'] == results['Predictions']) & (results['Predictions'] == 1.0)].shape

(11245, 5)

In [21]:
false_negatives = results[(results['True values'] != results['Predictions']) & (results['Predictions'] == 0.0)].copy()
print(false_negatives.shape[0])
false_negatives.head()

353


,Filenames,True values,Probas,Entry,Predictions
7828,human-true/human_true_A0A024A4F8_Envelope.png,1.0,0.020236,A0A024A4F8,0.0
7888,human-true/human_true_A0A024HR11_Autoprotease.png,1.0,0.107901,A0A024HR11,0.0
7971,human-true/human_true_A0A068BEH8_Togavirin.png,1.0,0.005496,A0A068BEH8,0.0
7979,human-true/human_true_A0A068F575_Core.png,1.0,0.086030,A0A068F575,0.0
8049,human-true/human_true_A0A075IIQ6_Hexon.png,1.0,0.180360,A0A075IIQ6,0.0


In [22]:
entries_false_negatives = df[df['Entry'].isin(false_negatives['Entry'].tolist())].copy()

In [23]:
entries_false_negatives['Protein'].nunique()

67

In [54]:
entries_false_negatives['Protein'].unique()

array(['Envelope polyprotein', 'Minor capsid protein L2', 'Protein BMRF2',
       'Minor capsid protein', 'Hexon-interlacing protein', 'Togavirin',
       'Pre-glycoprotein polyprotein GP complex', 'Spike glycoprotein',
       'Major capsid protein L1', 'Fiber', 'Outer capsid protein VP4',
       'Envelope glycoprotein gp160', 'Core protein',
       'Structural polyprotein', 'Envelope glycoprotein',
       'Assembly signal 2A', 'Glycoprotein', 'Capsid vertex component 2',
       'Core protein precursor', 'Gag polyprotein',
       'Envelope glycoprotein E1', 'Pre-histone-like nucleoprotein',
       'Capsid protein', 'Envelope glycoprotein C',
       'Gag-Pro-Pol polyprotein', 'Hemagglutinin-neuraminidase',
       'Genome polyprotein', 'Carbonic anhydrase homolog',
       'Envelope glycoprotein H', 'Coat protein',
       'Envelope glycoprotein B', '3D polymerase', 'Hexon protein',
       'Pre-protein VI', 'Exoribonuclease H', 'Fiber-1', 'Envelope',
       'Attachment glycoprotein', 'Prot

In [27]:
entries_false_negatives[(entries_false_negatives['Host kingdom'].str.contains("Viridiplantae"))]

,Entry,Protein,Length,Species name,Species taxonomic ID,Species family,Virus hosts,Virus hosts ID,Host kingdom,Host superkingdom,Molecule type,Infects human
44153,Q5GR62,Capsid protein,257.0,Honeysuckle yellow vein virus,240865,Geminiviridae,Lonicera [TaxID: 49606]; Lonicera japonica [Ta...,49606; 105884,Viridiplantae,Eukaryota,unknown,human-true
60321,Q80GK6,Capsid protein,257.0,Honeysuckle yellow vein mosaic virus,180587,Geminiviridae,Lonicera [TaxID: 49606],49606,Viridiplantae,Eukaryota,unknown,human-true


In [34]:
entries_false_positives['Virus hosts'].apply(lambda items: len(items.split("; "))).sort_values()

31        1
23079     1
23294     1
23969     1
23971     1
         ..
35184    14
14394    16
31990    19
31986    19
3536     66
Name: Virus hosts, Length: 259, dtype: int64

In [30]:
entries_false_negatives['Virus hosts'].apply(lambda items: len(items.split("; "))).sort_values()

58624     1
61867     1
62415     1
62471     1
62846     1
         ..
62930    14
85143    14
74831    14
69730    23
67881    23
Name: Virus hosts, Length: 353, dtype: int64

In [26]:
entries_false_negatives['Host kingdom'].unique()

array(['Metazoa', 'Viridiplantae'], dtype=object)

In [35]:
df[(df['Protein'].str.contains("gp160")) & (df['Infects human'].str.contains("true"))].shape

(12268, 12)

multiple hosts seem to cause confusion
due to labeling error during imputation as it was blind

In [53]:
entries_false_negatives['Virus hosts'].unique()

array(['Homo sapiens [TaxID: 9606]; Apodemus agrarius [TaxID: 39030]',
       'Homo sapiens [TaxID: 9606]',
       'Homo sapiens [TaxID: 9606]; Aedes [TaxID: 7158]; Passeriformes [TaxID: 9126]',
       'Homo sapiens [TaxID: 9606]; Aedes [TaxID: 7158]; Macropus sp. [TaxID: 9322]; Culex annulirostris [TaxID: 162997]',
       'Homo sapiens [TaxID: 9606]; Mastomys natalensis [TaxID: 10112]',
       'Homo sapiens [TaxID: 9606]; Paguma larvata [TaxID: 9675]; Rhinolophus macrotis [TaxID: 196889]; Rhinolophus sinicus [TaxID: 89399]; Rhinolophus ferrumequinum [TaxID: 59479]; Rhinolophus pearsonii [TaxID: 188571]',
       'Homo sapiens [TaxID: 9606]; Camelus dromedarius [TaxID: 9838]',
       'Macaca mulatta [TaxID: 9544]; Homo sapiens [TaxID: 9606]; Oryctolagus cuniculus [TaxID: 9986]; Canis lupus [TaxID: 9612]; Sus scrofa [TaxID: 9823]; Mus musculus [TaxID: 10090]; Bos taurus [TaxID: 9913]; Chlorocebus pygerythrus [TaxID: 60710]; Equus caballus [TaxID: 9796]',
       'Macaca mulatta [TaxID: 95

In [47]:
df[(df["Entry"].isin(results['Entry'].tolist())) & (df['Species name'].str.contains("Human immunodeficiency virus"))].shape

(2926, 12)

In [49]:
entries_false_negatives[entries_false_negatives['Species name'].str.contains("Human immunodeficiency virus")]

,Entry,Protein,Length,Species name,Species taxonomic ID,Species family,Virus hosts,Virus hosts ID,Host kingdom,Host superkingdom,Molecule type,Infects human
41733,A0A2P1DQ38,Envelope glycoprotein gp160,876.0,Simian-Human immunodeficiency virus,57667,Retroviridae,Macaca mulatta [TaxID: 9544]; Homo sapiens [Ta...,9545; 9606; 9544; 9539; 9541,Metazoa,Eukaryota,ssRNA-RT,human-true
43249,Q7SPP5,Gag polyprotein,1437.0,Human immunodeficiency virus 1,11676,Retroviridae,Homo sapiens [TaxID: 9606],9606,Metazoa,Eukaryota,ssRNA-RT,human-true
46935,A0A2P1DR91,Envelope glycoprotein gp160,858.0,Simian-Human immunodeficiency virus,57667,Retroviridae,Macaca mulatta [TaxID: 9544]; Homo sapiens [Ta...,9545; 9606; 9544; 9539; 9541,Metazoa,Eukaryota,ssRNA-RT,human-true
49810,A0A2D3HX09,Exoribonuclease H,876.0,Human immunodeficiency virus 2,11709,Retroviridae,Homo sapiens [TaxID: 9606],9606,Metazoa,Eukaryota,ssRNA-RT,human-true
51687,A0A2P1DR60,Envelope glycoprotein gp160,858.0,Simian-Human immunodeficiency virus,57667,Retroviridae,Macaca mulatta [TaxID: 9544]; Homo sapiens [Ta...,9545; 9606; 9544; 9539; 9541,Metazoa,Eukaryota,ssRNA-RT,human-true
51886,D2SS45,Protein Vpr,96.0,Human immunodeficiency virus 1,11676,Retroviridae,Homo sapiens [TaxID: 9606],9606,Metazoa,Eukaryota,ssRNA-RT,human-true
52829,A0A1D8BEG1,Envelope glycoprotein gp160,622.0,Human immunodeficiency virus 2,11709,Retroviridae,Homo sapiens [TaxID: 9606],9606,Metazoa,Eukaryota,ssRNA-RT,human-true
54685,G8J994,Envelope glycoprotein gp160,853.0,Human immunodeficiency virus 2,11709,Retroviridae,Homo sapiens [TaxID: 9606],9606,Metazoa,Eukaryota,ssRNA-RT,human-true
60614,Q5VG73,Envelope glycoprotein gp160,877.0,Human immunodeficiency virus 1,11676,Retroviridae,Homo sapiens [TaxID: 9606],9606,Metazoa,Eukaryota,ssRNA-RT,human-true
60689,L8B2L0,Envelope glycoprotein gp160,848.0,Human immunodeficiency virus 2,11709,Retroviridae,Homo sapiens [TaxID: 9606],9606,Metazoa,Eukaryota,ssRNA-RT,human-true


Labeling errors especially for the related viruses e.g. MERS related and SARS related viruses
Partial sequences being submitted esp for HIV, Human betaherpesvirus 6,
Viruses with sp. e.g. CRESS virus sp. could be strain issues so also attributatble to labeling errors

In [52]:
entries_false_negatives['Species name'].unique()

array(['Hantaan orthohantavirus', 'Human papillomavirus',
       'Human gammaherpesvirus 4', 'Human polyomavirus 8',
       'Human mastadenovirus A', 'Eastern equine encephalitis virus',
       'Ross River virus', 'Lassa mammarenavirus',
       'Severe acute respiratory syndrome-related coronavirus',
       'Alphapapillomavirus 3',
       'Middle East respiratory syndrome-related coronavirus',
       'Human mastadenovirus D', 'Rotavirus A',
       'Simian-Human immunodeficiency virus', 'Yellow fever virus',
       'Marburg marburgvirus', 'Hepatovirus A',
       'New Jersey vesiculovirus', 'Human alphaherpesvirus 1',
       'Hepacivirus C', 'Human immunodeficiency virus 1',
       'Venezuelan equine encephalitis virus', 'Pegivirus C',
       'Human mastadenovirus B', 'Honeysuckle yellow vein virus',
       'Alphapapillomavirus 7', 'Primate T-lymphotropic virus 3',
       'Avian metaavulavirus 2', 'Aichivirus A', 'Machupo mammarenavirus',
       'Cowpox virus', 'Human alphaherpesvirus 3'

In [80]:
entries_false_negatives['Length'].nsmallest(10)

86640     94.0
51886     96.0
78320     96.0
89637     96.0
70712     97.0
88936    100.0
83809    101.0
66878    102.0
69321    117.0
86593    117.0
Name: Length, dtype: float64

In [83]:
entries_false_positives['Length'].nsmallest(10)

35121     72.0
23014     95.0
35718     95.0
23714     96.0
33130     96.0
35954    100.0
32049    104.0
32572    104.0
17025    114.0
8205     117.0
Name: Length, dtype: float64

In [87]:
fns = entries_false_negatives['Entry'].tolist()
fps = entries_false_positives['Entry'].tolist()

In [88]:
fns.extend(fps)

In [89]:
len(fns)

612

In [52]:
df[df['Entry'].isin(results["Entry"].tolist())].sort_values("Length")

,Entry,Protein,Length,Species name,Species taxonomic ID,Species family,Virus hosts,Virus hosts ID,Host kingdom,Host superkingdom,Molecule type,Infects human
3796,A0A0E3HNZ7,Long tail fiber protein p37,51.0,Synechococcus virus AC2014fSyn7803C8,2734085,Myoviridae,Synechococcus [TaxID: 1129]; Synechococcus sp....,32051; 1129,Synechococcus sp. WH 7803; Synechococcus,Bacteria,unknown,human-false
32157,O72899,Protein I2 homolog,65.0,Fowlpox virus,10261,Poxviridae,Vertebrata [TaxID: 7742],7742,Metazoa,Eukaryota,unknown,human-false
27901,G8FRI5,Parvo_coat domain-containing protein,67.0,Carnivore protoparvovirus 1,1511906,Parvoviridae,Felidae [TaxID: 9681]; Canis lupus [TaxID: 961...,452646; 9654; 9612; 9685; 9651; 9681,Metazoa,Eukaryota,ssDNA(+/-),human-false
90077,A0A7G7XMH8,HDAg-S,70.0,Hepatitis delta virus,12475,Kolmioviridae,Homo sapiens [TaxID: 9606],9606,Metazoa,Eukaryota,ssRNA(-),human-true
35121,Q89707,Pre-histone-like nucleoprotein,72.0,Fowl aviadenovirus A,190061,Adenoviridae,Galliformes [TaxID: 8976],8976,Metazoa,Eukaryota,unknown,human-false
...,...,...,...,...,...,...,...,...,...,...,...,...
29110,I6QIZ9,Autoprotease p20,4001.0,Pestivirus H,2170087,Flaviviridae,Capra hircus [TaxID: 9925]; Ovis aries [TaxID:...,89462; 9913; 9915; 9925; 9940,Metazoa,Eukaryota,ssRNA(+),human-false
81496,A0A3S9KG07,Autoprotease p20,4028.0,Pestivirus B,2170081,Flaviviridae,Homo sapiens [TaxID: 9606]; Capra hircus [TaxI...,89462; 9823; 30521; 9895; 9913; 9606; 9925; 9940,Metazoa,Eukaryota,ssRNA(+),human-true
35723,Q9IFH8,Autoprotease p20,4040.0,Pestivirus A,2170080,Flaviviridae,Bos taurus [TaxID: 9913],9913,Metazoa,Eukaryota,ssRNA(+),human-false
34831,Q807Z8,Autoprotease p20,4197.0,Pestivirus A,2170080,Flaviviridae,Bos taurus [TaxID: 9913],9913,Metazoa,Eukaryota,ssRNA(+),human-false


length does not seem to have been a factor as some of the correctly classified values were relatively small sequences between 34 and 71 amino acids.